# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup

## Data read-in

In [2]:
# df = pd.read_clipboard()
# df.to_csv("new_sales_listings.csv")

In [3]:
df = pd.read_csv("Brandon Miller - Sheet1.csv")

## Geocode

In [5]:
df['full_address'] = df['Address'] + ' ' + df['Nabe'] + ' NY'

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [7]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,Project,Address,Nabe,Acris link,LLC,State court,Federal court,Clips,Floors,SF,Units,Unnamed: 11,full_address,geocoded,lat,lon,geometry
0,New office building (leasehold),3 St Marks Place,East Village,http://a836-acris.nyc.gov/bblsearch/bblsearch....,REEC ST MARKS LESSEE LP\n3 SAINT MARKS,https://iapps.courts.state.ny.us/nyscef/Docume...,NaN,https://therealdeal.com/new-york/2022/06/10/1-...,9.0,"60,528",NaN,NaN,3 St Marks Place East Village NY,"(40.7295381, -73.9893618)",40.729538,-73.989362,POINT (-73.98936 40.72954)
1,New office building (leasehold),118 TENTH AVENUE,Chelsea/Meat Packing,http://a836-acris.nyc.gov/bblsearch/bblsearch....,REEC 118 10TH AVENUE,NaN,NaN,NaN,10.0,"100,000",NaN,sold to GDS Development in 2019,118 TENTH AVENUE Chelsea/Meat Packing NY,"(40.7441775, -74.00628499999999)",40.744177,-74.006285,POINT (-74.00628 40.74418)
2,New life science building partnered with Night...,2226 3 AVENUE,East Harlem,http://a836-acris.nyc.gov/bblsearch/bblsearch....,REEC THIRD AVE,NaN,leashold,NaN,NaN,NaN,NaN,NaN,2226 3 AVENUE East Harlem NY,"(40.808487, -73.9314989)",40.808487,-73.931499,POINT (-73.93150 40.80849)
3,Bank foreclosure 1/2023,313-315 West 125th Street,Central Harlem,https://a836-acris.nyc.gov/DS/DocumentSearch/B...,NaN,NaN,NaN,REEC WEST 125TH STREET LLC,NaN,NaN,leasehold/estate of Sol Goldman is fee owner,https://www.pincusco.com/reec-was-partner-with...,313-315 West 125th Street Central Harlem NY,"(40.8108017, -73.95182539999999)",40.810802,-73.951825,POINT (-73.95183 40.81080)
4,New office building (leasehold),286 LENOX AVENUE,Central Harlem,http://a836-acris.nyc.gov/bblsearch/bblsearch....,NaN,NaN,NaN,https://www.bossequities.com/286-lenox-ave-new...,NaN,NaN,NaN,NaN,286 LENOX AVENUE Central Harlem NY,"(40.8072104, -73.9454829)",40.807210,-73.945483,POINT (-73.94548 40.80721)
5,Luxury condos,1228 madison avenue,Upper East Side,http://a836-acris.nyc.gov/bblsearch/bblsearch....,1228-1234 MADISON AVENUE LLC\n1228 MADISON DEV...,https://iapps.courts.state.ny.us/nyscef/Docume...,NaN,NaN,17.0,"60,000",15,** looks like REEC gout out of this project in...,1228 madison avenue Upper East Side NY,"(40.7825765, -73.9578338)",40.782576,-73.957834,POINT (-73.95783 40.78258)
6,Condos,196 Orchard Street,Lower East Side,NaN,MB-REEC HOUSTON PROPERTY OWNER,NaN,NaN,NaN,94.0,"149,212",94,Sold to NYC Reit https://therealdeal.com/new-y...,196 Orchard Street Lower East Side NY,"(40.722264, -73.987985)",40.722264,-73.987985,POINT (-73.98798 40.72226)
7,Times Square Retail - Long play for new dev,1604 Broadway,Times Square,http://a836-acris.nyc.gov/bblsearch/bblsearch....,"1604 BROADWAY HOLDER, LP",NaN,NaN,https://www.crainsnewyork.com/real-estate/time...,NaN,NaN,NaN,NaN,1604 Broadway Times Square NY,"(40.7579747, -73.9855426)",40.757975,-73.985543,POINT (-73.98554 40.75797)
9,bought leasehold in 2020 for $50M,162-66 Bowery,Lower East Side,https://a836-acris.nyc.gov/DS/DocumentSearch/B...,REEC 156 BOWERY,NaN,NaN,60.5M loan from Raven Capital in 2022. Plans f...,NaN,NaN,NaN,NaN,162-66 Bowery Lower East Side NY,"(40.7201299, -73.9944347)",40.720130,-73.994435,POINT (-73.99443 40.72013)
10,"Acquired in February of 2015, the asset stands...",576 fifth ave,https://a836-acris.nyc.gov/DS/DocumentSearch/D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,576 fifth ave https://a836-acris.nyc.gov/DS/Do...,"(40.7566502, -73.9790192)",40.756650,-73.979019,POINT (-73.97902 40.75665)


In [8]:
gdf

,Project,Address,Nabe,Acris link,LLC,State court,Federal court,Clips,Floors,SF,Units,Unnamed: 11,full_address,geocoded,lat,lon,geometry
0,New office building (leasehold),3 St Marks Place,East Village,http://a836-acris.nyc.gov/bblsearch/bblsearch....,REEC ST MARKS LESSEE LP\n3 SAINT MARKS,https://iapps.courts.state.ny.us/nyscef/Docume...,NaN,https://therealdeal.com/new-york/2022/06/10/1-...,9.0,"60,528",NaN,NaN,3 St Marks Place East Village NY,"(40.7295381, -73.9893618)",40.729538,-73.989362,POINT (-73.98936 40.72954)
1,New office building (leasehold),118 TENTH AVENUE,Chelsea/Meat Packing,http://a836-acris.nyc.gov/bblsearch/bblsearch....,REEC 118 10TH AVENUE,NaN,NaN,NaN,10.0,"100,000",NaN,sold to GDS Development in 2019,118 TENTH AVENUE Chelsea/Meat Packing NY,"(40.7441775, -74.00628499999999)",40.744177,-74.006285,POINT (-74.00628 40.74418)
2,New life science building partnered with Night...,2226 3 AVENUE,East Harlem,http://a836-acris.nyc.gov/bblsearch/bblsearch....,REEC THIRD AVE,NaN,leashold,NaN,NaN,NaN,NaN,NaN,2226 3 AVENUE East Harlem NY,"(40.808487, -73.9314989)",40.808487,-73.931499,POINT (-73.93150 40.80849)
3,Bank foreclosure 1/2023,313-315 West 125th Street,Central Harlem,https://a836-acris.nyc.gov/DS/DocumentSearch/B...,NaN,NaN,NaN,REEC WEST 125TH STREET LLC,NaN,NaN,leasehold/estate of Sol Goldman is fee owner,https://www.pincusco.com/reec-was-partner-with...,313-315 West 125th Street Central Harlem NY,"(40.8108017, -73.95182539999999)",40.810802,-73.951825,POINT (-73.95183 40.81080)
4,New office building (leasehold),286 LENOX AVENUE,Central Harlem,http://a836-acris.nyc.gov/bblsearch/bblsearch....,NaN,NaN,NaN,https://www.bossequities.com/286-lenox-ave-new...,NaN,NaN,NaN,NaN,286 LENOX AVENUE Central Harlem NY,"(40.8072104, -73.9454829)",40.807210,-73.945483,POINT (-73.94548 40.80721)
5,Luxury condos,1228 madison avenue,Upper East Side,http://a836-acris.nyc.gov/bblsearch/bblsearch....,1228-1234 MADISON AVENUE LLC\n1228 MADISON DEV...,https://iapps.courts.state.ny.us/nyscef/Docume...,NaN,NaN,17.0,"60,000",15,** looks like REEC gout out of this project in...,1228 madison avenue Upper East Side NY,"(40.7825765, -73.9578338)",40.782576,-73.957834,POINT (-73.95783 40.78258)
6,Condos,196 Orchard Street,Lower East Side,NaN,MB-REEC HOUSTON PROPERTY OWNER,NaN,NaN,NaN,94.0,"149,212",94,Sold to NYC Reit https://therealdeal.com/new-y...,196 Orchard Street Lower East Side NY,"(40.722264, -73.987985)",40.722264,-73.987985,POINT (-73.98798 40.72226)
7,Times Square Retail - Long play for new dev,1604 Broadway,Times Square,http://a836-acris.nyc.gov/bblsearch/bblsearch....,"1604 BROADWAY HOLDER, LP",NaN,NaN,https://www.crainsnewyork.com/real-estate/time...,NaN,NaN,NaN,NaN,1604 Broadway Times Square NY,"(40.7579747, -73.9855426)",40.757975,-73.985543,POINT (-73.98554 40.75797)
9,bought leasehold in 2020 for $50M,162-66 Bowery,Lower East Side,https://a836-acris.nyc.gov/DS/DocumentSearch/B...,REEC 156 BOWERY,NaN,NaN,60.5M loan from Raven Capital in 2022. Plans f...,NaN,NaN,NaN,NaN,162-66 Bowery Lower East Side NY,"(40.7201299, -73.9944347)",40.720130,-73.994435,POINT (-73.99443 40.72013)
10,"Acquired in February of 2015, the asset stands...",576 fifth ave,https://a836-acris.nyc.gov/DS/DocumentSearch/D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,576 fifth ave https://a836-acris.nyc.gov/DS/Do...,"(40.7566502, -73.9790192)",40.756650,-73.979019,POINT (-73.97902 40.75665)


In [9]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [11]:
gdf.to_file('out_put_brandon_miller_properties.geojson', driver='GeoJSON')

In [12]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NYC_Brandon_Miller_09_20_24


<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>

In [13]:
gdf.columns

Index(['Project', 'Address', 'Nabe', 'Acris link', 'LLC', 'State court',
       'Federal court', 'Clips', 'Floors', 'SF', 'Units', 'Unnamed: 11',
       'full_address', 'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')